# Experiment and indicator calculation

Notice for Reviewing This Notebook
This notebook contains Step 1–5 results for both datasets (Adult & Kaggle).
Due to the large memory requirements in Step 5, please do not click “Run all”.
Instead:

All results have already been executed and displayed in the output cells.

Scroll down to see the printed tables (raw results, mean±std summaries, significance tests).


If re-running is necessary, we recommend executing Step 5A (Adult) and Step 5B (Kaggle) separately to avoid RAM overflow.

In [ ]:
!pip -q install aif360==0.6.1 folktables==0.0.12 imbalanced-learn==0.12.3
!pip -q install scikit-learn==1.4.2 pandas==2.2.2 numpy==1.26.4 scipy==1.13.1
!pip -q install tensorflow==2.15.0
!pip -q install "aif360[Reductions]" "aif360[inFairness]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires sci

In [ ]:
import os, signal, sys, time
os.kill(os.getpid(), signal.SIGKILL)


In [ ]:
import numpy as np, pandas as pd, random
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from aif360.datasets import BinaryLabelDataset, AdultDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.metrics import ClassificationMetric

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

SEED = 42
np.random.seed(SEED); random.seed(SEED)


/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving recruitment_data.csv to recruitment_data (1).csv
Saving adult_test.csv to adult_test (1).csv
Saving adult_train.csv to adult_train (1).csv


In [ ]:
train_df = pd.read_csv("adult_train.csv")
test_df = pd.read_csv("adult_test.csv")
kaggle_df = pd.read_csv("recruitment_data.csv")

print("Training set size:", train_df.shape)
print("Test set size:", test_df.shape)
print("Kaggle set size:", kaggle_df.shape)

# Step 1: Data inspection and light normalization

# 1.1 Basic overview
print('\n[Columns] train:', train_df.columns.tolist())
print('[Columns] test :', test_df.columns.tolist())
print('[Columns] kaggle:', kaggle_df.columns.tolist())

print('\n[Head] Adult train:')
display(train_df.head(3))
print('\n[Head] Adult test:')
display(test_df.head(3))
print('\n[Head] Kaggle:')
display(kaggle_df.head(3))

# 1.2 Quick statistics & label distribution
for name, df in [('Adult train', train_df), ('Adult test', test_df)]:
    if 'income' in df.columns:
        print(f'\n[{name}] income value_counts:')
        print(df['income'].value_counts(dropna=False))
    else:
        print(f'\n[{name}] No income column found, please confirm label column name')

# 1.3 Remove leading/trailing spaces in string columns
def strip_object_cols(df):
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = out[c].astype(str).str.strip()
    return out

train_df = strip_object_cols(train_df)
test_df  = strip_object_cols(test_df)
kaggle_df = strip_object_cols(kaggle_df)

# 1.4 Replace '?' and empty strings with NaN for consistent missing value handling
replace_map = {'?': np.nan, '': np.nan, ' ': np.nan}
train_df = train_df.replace(replace_map)
test_df  = test_df.replace(replace_map)
kaggle_df = kaggle_df.replace(replace_map)

# 1.5 Re-check income distribution in test set
print('\n[After strip/replace] Adult test income value_counts:')
if 'income' in test_df.columns:
    print(test_df['income'].value_counts(dropna=False))
else:
    print('No income column found in Adult test set')

print('Step 1 complete.')




Training set size: (32561, 15)
Test set size: (16280, 15)
Kaggle set size: (1500, 11)

[Columns] train: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
[Columns] test : ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
[Columns] kaggle: ['Age', 'Gender', 'EducationLevel', 'ExperienceYears', 'PreviousCompanies', 'DistanceFromCompany', 'InterviewScore', 'SkillScore', 'PersonalityScore', 'RecruitmentStrategy', 'HiringDecision']

[Head] Adult train:


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K



[Head] Adult test:


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
1,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
2,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K



[Head] Kaggle:


,Age,Gender,EducationLevel,ExperienceYears,PreviousCompanies,DistanceFromCompany,InterviewScore,SkillScore,PersonalityScore,RecruitmentStrategy,HiringDecision
0,26,1,2,0,3,26.783828,48,78,91,1,1
1,39,1,4,12,3,25.862694,35,68,80,2,1
2,48,0,2,3,2,9.920805,20,67,13,2,0



[Adult train] income value_counts:
income
<=50K    24720
>50K      7841
Name: count, dtype: int64

[Adult test] income value_counts:
income
<=50K    12434
>50K      3846
Name: count, dtype: int64

[After strip/replace] Adult test income value_counts:
income
<=50K    12434
>50K      3846
Name: count, dtype: int64
Step 1 complete.


In [ ]:
# Step 2: Full cleaning & leakage prevention


# 2.1 Helpers

def strip_object_cols(df):
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = out[c].astype(str).str.strip()
    return out


def clean_adult_df(df: pd.DataFrame,
                   label_col: str = 'income',
                   age_thr: int = 40,
                   edu_thr: int = 13) -> pd.DataFrame:
    """Standardize Adult: strip spaces, unify label to 0/1, mark NaNs, cast numeric,
       and derive sensitive attributes: age_bin, edu_bin."""
    out = strip_object_cols(df)

    # Replace common missing markers with NaN
    out = out.replace({'?': np.nan, '': np.nan, ' ': np.nan})

    # Label -> numeric {<=50K:0, >50K:1} (support dotted variants if present)
    if label_col not in out.columns:
        raise ValueError(f"Label column '{label_col}' not found in Adult dataset.")
    if out[label_col].dtype == object:
        lab_map = {'>50K':1, '<=50K':0, '>50K.':1, '<=50K.':0}
        out['label'] = out[label_col].map(lab_map).astype('Int64')
    else:
        out['label'] = out[label_col].astype(int)

    # Cast likely numeric columns
    for c in ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors='coerce')

    # Derive sensitive attributes (for fairness eval & debiasing constraints, not for model input)
    out['age_bin'] = (out['age'] < age_thr).astype('Int64') if 'age' in out.columns else pd.Series([pd.NA]*len(out), dtype='Int64')
    if 'education-num' in out.columns:
        out['edu_bin'] = (pd.to_numeric(out['education-num'], errors='coerce') < edu_thr).astype('Int64')
    else:
        out['edu_bin'] = pd.Series([pd.NA]*len(out), dtype='Int64')

    return out

def clean_kaggle_df(df: pd.DataFrame,
                    label_col: str = 'HiringDecision',
                    age_col: str = 'Age',
                    edu_col: str = 'EducationLevel',
                    age_thr: int = 40,
                    edu_low_max: int = 2) -> pd.DataFrame:
    """Standardize Kaggle recruitment: strip spaces, unify label to 0/1,
       cast numeric, map ordered education, and derive sensitive bins."""
    out = strip_object_cols(df)
    out = out.replace({'?': np.nan, '': np.nan, ' ': np.nan})

    # Label -> numeric 0/1
    if label_col not in out.columns:
        raise ValueError(f"Label column '{label_col}' not found in Kaggle dataset.")
    out['label'] = pd.to_numeric(out[label_col], errors='coerce').astype('Int64')

    # Cast likely numeric columns (extend if your CSV has more)
    numeric_like = ['Age','ExperienceYears','PreviousCompanies','DistanceFromCompany',
                    'InterviewScore','SkillScore','PersonalityScore']
    for c in numeric_like:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors='coerce')

    # Map ordered education if it's categorical
    if edu_col in out.columns:
        if out[edu_col].dtype == object:
            edu_map = {'HighSchool':1, 'Diploma':2, 'Bachelors':3, 'Masters':4, 'PhD':5}
            out['education_ord'] = out[edu_col].map(edu_map).astype('Int64')
        else:
            out['education_ord'] = pd.to_numeric(out[edu_col], errors='coerce').astype('Int64')
    else:
        out['education_ord'] = pd.Series([pd.NA]*len(out), dtype='Int64')

    # Sensitive attributes
    if age_col in out.columns:
        out['age_bin'] = (pd.to_numeric(out[age_col], errors='coerce') < age_thr).astype('Int64')
    else:
        out['age_bin'] = pd.Series([pd.NA]*len(out), dtype='Int64')

    # Low education protected group (<= Diploma)
    out['edu_bin'] = (out['education_ord'] <= edu_low_max).astype('Int64')

    return out

def detect_leak_columns(df: pd.DataFrame,
                        label: str = 'label',
                        base_drop: list = None,
                        max_unique: int = 10,
                        corr_thresh: float = 0.999) -> list:
    """Detect suspicious leak features: near-perfect correlation with label (numeric)
       or perfect separation (categorical with small cardinality)."""
    if base_drop is None:
        base_drop = []
    leak_cols = set()
    y = pd.to_numeric(df[label], errors='coerce').astype(float)

    for c in df.columns:
        if c in base_drop or c == label:
            continue
        s = df[c]
        # Numeric correlation near ±1
        if s.dtype.kind in 'ifu':
            s = pd.to_numeric(s, errors='coerce')
            corr = s.corr(y)
            if pd.notna(corr) and abs(corr) >= corr_thresh:
                leak_cols.add(c)
        else:
            # Categorical perfect partition: each category maps to a single label
            nunique = s.nunique(dropna=True)
            if nunique <= max_unique and nunique > 0:
                tab = df[[c, label]].dropna().groupby(c)[label].nunique()
                if len(tab) > 0 and (tab <= 1).all():
                    leak_cols.add(c)
    return sorted(list(leak_cols))

def check_train_test_overlap(train_df: pd.DataFrame, test_df: pd.DataFrame) -> int:
    """Rough overlap check by hashing rows (after sorting columns)."""
    common_cols = sorted(set(train_df.columns) & set(test_df.columns))
    if not common_cols:
        return 0
    t_hash = pd.util.hash_pandas_object(train_df[common_cols], index=False)
    e_hash = pd.util.hash_pandas_object(test_df[common_cols], index=False)
    overlap = np.intersect1d(t_hash.values, e_hash.values).size
    return int(overlap)

# 2.2 Clean datasets

# Adult
train_df = clean_adult_df(train_df, label_col='income', age_thr=40, edu_thr=13)
test_df  = clean_adult_df(test_df,  label_col='income', age_thr=40, edu_thr=13)

# Kaggle
kaggle_df = clean_kaggle_df(kaggle_df,
                            label_col='HiringDecision',
                            age_col='Age',
                            edu_col='EducationLevel',
                            age_thr=40,
                            edu_low_max=2)

# 2.3 Leakage prevention setup

adult_base_drop = ['label', 'age_bin', 'edu_bin', 'income']
kaggle_base_drop = ['label', 'age_bin', 'edu_bin', 'education_ord', 'HiringDecision']

# Detect suspicious leak columns
adult_leaks = detect_leak_columns(train_df, label='label', base_drop=adult_base_drop)
kaggle_leaks = detect_leak_columns(kaggle_df, label='label', base_drop=kaggle_base_drop)

print('Adult suspicious leak columns:', adult_leaks if adult_leaks else 'None')
print('Kaggle suspicious leak columns:', kaggle_leaks if kaggle_leaks else 'None')

# 2.4 Train/Test overlap check for Adult
overlap_n = check_train_test_overlap(train_df, test_df)
print(f'Adult train/test potential row overlap (hash-based): {overlap_n}')

# 2.5 Quick QC summaries

def qc_summary(name: str, df: pd.DataFrame, label_col='label'):
    print(f'\n--- {name} QC ---')
    if label_col in df.columns:
        print('Label distribution:')
        print(df[label_col].value_counts(dropna=False))
    misses = df.isna().sum().sort_values(ascending=False).head(10)
    print('\nTop-10 columns by missing values:')
    print(misses)

qc_summary('Adult TRAIN', train_df)
qc_summary('Adult TEST', test_df)
qc_summary('Kaggle (raw, before split)', kaggle_df)

print('Step 2 done')


Adult suspicious leak columns: None
Kaggle suspicious leak columns: None
Adult train/test potential row overlap (hash-based): 23

--- Adult TRAIN QC ---
Label distribution:
label
0    24720
1     7841
Name: count, dtype: Int64

Top-10 columns by missing values:
age               0
workclass         0
age_bin           0
label             0
income            0
native-country    0
hours-per-week    0
capital-loss      0
capital-gain      0
sex               0
dtype: int64

--- Adult TEST QC ---
Label distribution:
label
0    12434
1     3846
Name: count, dtype: Int64

Top-10 columns by missing values:
age               0
workclass         0
age_bin           0
label             0
income            0
native-country    0
hours-per-week    0
capital-loss      0
capital-gain      0
sex               0
dtype: int64

--- Kaggle (raw, before split) QC ---
Label distribution:
label
0    1035
1     465
Name: count, dtype: Int64

Top-10 columns by missing values:
Age                    0
Gender   

In [ ]:

# Step 3: Feature engineering + Baseline logistic regression


# 3.0 Utility: drop overlapping rows in Adult test (hash-based)
def drop_overlaps(train_df, test_df):
    common_cols = sorted(set(train_df.columns) & set(test_df.columns))
    if not common_cols:
        return test_df, 0
    tr_hash = pd.util.hash_pandas_object(train_df[common_cols], index=False)
    te_hash = pd.util.hash_pandas_object(test_df[common_cols], index=False)
    mask_keep = ~test_df.index.isin(test_df.index[np.where(te_hash.isin(tr_hash))[0]])
    removed = (~mask_keep).sum()
    return test_df[mask_keep].reset_index(drop=True), int(removed)

# 3.1 Prepare Adult features
adult_base_drop = ['label', 'age_bin', 'edu_bin', 'income']
adult_leaks = []

adult_drop_cols = adult_base_drop + adult_leaks
adult_X_cols = [c for c in train_df.columns if c not in adult_drop_cols]

adult_num_cols = [c for c in adult_X_cols if train_df[c].dtype.kind in 'if']
adult_cat_cols = [c for c in adult_X_cols if c not in adult_num_cols]

adult_preprocess = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imp', SimpleImputer(strategy='median')),
            ('sc', StandardScaler())
        ]), adult_num_cols),
        ('cat', Pipeline(steps=[
            ('imp', SimpleImputer(strategy='most_frequent')),
            ('oh', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), adult_cat_cols)
    ],
    remainder='drop'
)

# Drop overlaps in Adult test
adult_test_dedup, removed_n = drop_overlaps(train_df, test_df)
print(f'[Adult] Removed {removed_n} overlapping rows from test set (if any).')

# Fit transform Adult
X_tr_ad = train_df[adult_X_cols].copy()
y_tr_ad = train_df['label'].astype(int).values
X_te_ad = adult_test_dedup[adult_X_cols].copy()
y_te_ad = adult_test_dedup['label'].astype(int).values

X_tr_ad_t = adult_preprocess.fit_transform(X_tr_ad)
X_te_ad_t = adult_preprocess.transform(X_te_ad)

# Train baseline model (Adult)
clf_ad = LogisticRegression(max_iter=300, random_state=SEED)
clf_ad.fit(X_tr_ad_t, y_tr_ad)
y_score_ad = clf_ad.predict_proba(X_te_ad_t)[:, 1]
y_pred_ad  = (y_score_ad >= 0.5).astype(int)

perf_ad = {
    'accuracy': accuracy_score(y_te_ad, y_pred_ad),
    'f1': f1_score(y_te_ad, y_pred_ad),
    'auc': roc_auc_score(y_te_ad, y_score_ad)
}
print('[Adult] Baseline performance:', perf_ad)

# Adult fairness metrics (age, edu)
def to_bld(X_np, y, prot_name, prot_vals):
    df_tmp = pd.DataFrame(X_np)
    df_tmp['label'] = y
    df_tmp[prot_name] = prot_vals
    return BinaryLabelDataset(
        df=df_tmp,
        label_names=['label'],
        protected_attribute_names=[prot_name],
        favorable_label=1, unfavorable_label=0
    )

def fairness_report(bld_true, bld_pred, prot_name, unpriv=1, priv=0):
    cm = ClassificationMetric(
        bld_true, bld_pred,
        unprivileged_groups=[{prot_name: unpriv}],
        privileged_groups=[{prot_name: priv}]
    )
    return {
        'SPD': cm.statistical_parity_difference(),
        'EOD': cm.equal_opportunity_difference(),
        'DI':  cm.disparate_impact()
    }

p_te_ad_age = adult_test_dedup['age_bin'].astype('Int64').fillna(0).astype(int).values
p_te_ad_edu = adult_test_dedup['edu_bin'].astype('Int64').fillna(0).astype(int).values

bld_te_age_ad   = to_bld(X_te_ad_t, y_te_ad, 'age_bin', p_te_ad_age)
bld_pred_age_ad = to_bld(X_te_ad_t, y_pred_ad, 'age_bin', p_te_ad_age)
fair_age_ad = fairness_report(bld_te_age_ad, bld_pred_age_ad, 'age_bin')
print('[Adult] Fairness (Age):', fair_age_ad)

bld_te_edu_ad   = to_bld(X_te_ad_t, y_te_ad, 'edu_bin', p_te_ad_edu)
bld_pred_edu_ad = to_bld(X_te_ad_t, y_pred_ad, 'edu_bin', p_te_ad_edu)
fair_edu_ad = fairness_report(bld_te_edu_ad, bld_pred_edu_ad, 'edu_bin')
print('[Adult] Fairness (Education):', fair_edu_ad)

# 3.2 Kaggle: stratified train/test split, build pipeline, train baseline
kaggle_base_drop = ['label', 'age_bin', 'edu_bin', 'education_ord', 'HiringDecision']
kaggle_leaks = []
kaggle_drop_cols = kaggle_base_drop + kaggle_leaks

kaggle_X_cols = [c for c in kaggle_df.columns if c not in kaggle_drop_cols]
kaggle_num_cols = [c for c in kaggle_X_cols if kaggle_df[c].dtype.kind in 'if']
kaggle_cat_cols = [c for c in kaggle_X_cols if c not in kaggle_num_cols]

kaggle_preprocess = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imp', SimpleImputer(strategy='median')),
            ('sc', StandardScaler())
        ]), kaggle_num_cols),
        ('cat', Pipeline(steps=[
            ('imp', SimpleImputer(strategy='most_frequent')),
            ('oh', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), kaggle_cat_cols)
    ],
    remainder='drop'
)

# Split Kaggle
tr_kg, te_kg = train_test_split(
    kaggle_df, test_size=0.2, random_state=SEED, stratify=kaggle_df['label']
)

X_tr_kg = tr_kg[kaggle_X_cols].copy(); y_tr_kg = tr_kg['label'].astype(int).values
X_te_kg = te_kg[kaggle_X_cols].copy(); y_te_kg = te_kg['label'].astype(int).values

X_tr_kg_t = kaggle_preprocess.fit_transform(X_tr_kg)
X_te_kg_t = kaggle_preprocess.transform(X_te_kg)

# Train baseline (Kaggle)
clf_kg = LogisticRegression(max_iter=300, random_state=SEED)
clf_kg.fit(X_tr_kg_t, y_tr_kg)
y_score_kg = clf_kg.predict_proba(X_te_kg_t)[:, 1]
y_pred_kg  = (y_score_kg >= 0.5).astype(int)

perf_kg = {
    'accuracy': accuracy_score(y_te_kg, y_pred_kg),
    'f1': f1_score(y_te_kg, y_pred_kg),
    'auc': roc_auc_score(y_te_kg, y_score_kg)
}
print('[Kaggle] Baseline performance:', perf_kg)

# Kaggle fairness (age & edu)
p_te_kg_age = te_kg['age_bin'].astype('Int64').fillna(0).astype(int).values
p_te_kg_edu = te_kg['edu_bin'].astype('Int64').fillna(0).astype(int).values

bld_te_age_kg   = to_bld(X_te_kg_t, y_te_kg, 'age_bin', p_te_kg_age)
bld_pred_age_kg = to_bld(X_te_kg_t, y_pred_kg, 'age_bin', p_te_kg_age)
fair_age_kg = fairness_report(bld_te_age_kg, bld_pred_age_kg, 'age_bin')
print('[Kaggle] Fairness (Age):', fair_age_kg)

bld_te_edu_kg   = to_bld(X_te_kg_t, y_te_kg, 'edu_bin', p_te_kg_edu)
bld_pred_edu_kg = to_bld(X_te_kg_t, y_pred_kg, 'edu_bin', p_te_kg_edu)
fair_edu_kg = fairness_report(bld_te_edu_kg, bld_pred_edu_kg, 'edu_bin')
print('[Kaggle] Fairness (Education):', fair_edu_kg)

print('Step 3 complete.')


[Adult] Removed 23 overlapping rows from test set (if any).
[Adult] Baseline performance: {'accuracy': 0.852924893891862, 'f1': 0.657988842797883, 'auc': 0.9053385929054687}
[Adult] Fairness (Age): {'SPD': -0.18588232223274836, 'EOD': -0.12899178728284322, 'DI': 0.3757110369686625}
[Adult] Fairness (Education): {'SPD': -0.43415169294206646, 'EOD': -0.4791272424546021, 'DI': 0.16520427217047143}
[Kaggle] Baseline performance: {'accuracy': 0.8566666666666667, 'f1': 0.7514450867052023, 'auc': 0.8911225390888785}
[Kaggle] Fairness (Age): {'SPD': 0.006030453791647794, 'EOD': -0.050793650793650724, 'DI': 1.0229621125143513}
[Kaggle] Fairness (Education): {'SPD': -0.2644087938205585, 'EOD': -0.16187384044526898, 'DI': 0.4006734006734007}
Step 3 complete.


In [ ]:
# Step 4: Debiasing methods comparison (Reweighing / Adversarial Debiasing / Reject Option Classification)



# 4.0 Helpers

def compute_performance(y_true, y_pred, y_score):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc': roc_auc_score(y_true, y_score) if len(np.unique(y_true)) == 2 else np.nan
    }

def to_bld_from_arrays(X, y, prot, prot_name='prot'):
    """Build an AIF360 BinaryLabelDataset from numpy arrays (features + label + protected)."""

    cols = [f'f{i}' for i in range(X.shape[1])]
    df = pd.DataFrame(X, columns=cols)
    df['label'] = y
    df[prot_name] = prot
    bld = BinaryLabelDataset(
        df=df,
        label_names=['label'],
        protected_attribute_names=[prot_name],
        favorable_label=1,
        unfavorable_label=0
    )
    return bld

def fairness_report(bld_true, bld_pred, prot_name, unpriv=1, priv=0):
    cm = ClassificationMetric(
        bld_true, bld_pred,
        unprivileged_groups=[{prot_name: unpriv}],
        privileged_groups=[{prot_name: priv}]
    )
    return {
        'SPD': cm.statistical_parity_difference(),
        'EOD': cm.equal_opportunity_difference(),
        'DI':  cm.disparate_impact()
    }

def print_block(title, obj):
    print('\n' + '='*90)
    print(title)
    print('='*90)
    print(obj)

# 4.1 Prepare protected attributes (train & test) for both datasets

# Adult train protected attributes
p_tr_ad_age = train_df['age_bin'].astype('Int64').fillna(0).astype(int).values
p_tr_ad_edu = train_df['edu_bin'].astype('Int64').fillna(0).astype(int).values

# Adult test protected attributes
# Kaggle train/test protected attributes
p_tr_kg_age = tr_kg['age_bin'].astype('Int64').fillna(0).astype(int).values
p_tr_kg_edu = tr_kg['edu_bin'].astype('Int64').fillna(0).astype(int).values
p_te_kg_age = te_kg['age_bin'].astype('Int64').fillna(0).astype(int).values
p_te_kg_edu = te_kg['edu_bin'].astype('Int64').fillna(0).astype(int).values

# 4.2 REWEIGHING (pre-processing)
# Train logistic regression using instance weights produced by Reweighing.

def run_reweighing(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name):

    bld_tr = to_bld_from_arrays(X_tr, y_tr, prot_tr, prot_name=prot_name)
    rw = Reweighing(
        unprivileged_groups=[{prot_name: 1}],
        privileged_groups=[{prot_name: 0}]
    )
    bld_tr_rw = rw.fit_transform(bld_tr)
    sample_w = bld_tr_rw.instance_weights


    clf = LogisticRegression(max_iter=300, random_state=SEED)
    clf.fit(X_tr, y_tr, sample_weight=sample_w)
    y_score = clf.predict_proba(X_te)[:, 1]
    y_pred = (y_score >= 0.5).astype(int)


    bld_te_true = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_pred = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te_true, bld_te_pred, prot_name)
    return perf, fair

# 4.3 ADVERSARIAL DEBIASING (in-processing)
# Train a debiased classifier in TensorFlow via AIF360.

def run_adversarial(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, scope):
    bld_tr = to_bld_from_arrays(X_tr, y_tr, prot_tr, prot_name=prot_name)
    bld_te = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)

    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    np.random.seed(SEED)
    tf.compat.v1.set_random_seed(SEED)

    debias = AdversarialDebiasing(
        privileged_groups=[{prot_name: 0}],
        unprivileged_groups=[{prot_name: 1}],
        scope_name=scope,
        debias=True,
        sess=sess,
        num_epochs=50,
        batch_size=256,
        classifier_num_hidden_units=64
    )
    debias.fit(bld_tr)


    bld_pred = debias.predict(bld_te)


    y_pred = bld_pred.labels.ravel().astype(int)

    y_score = bld_pred.scores.ravel()

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te, bld_pred, prot_name)

    sess.close()
    return perf, fair

# 4.4 REJECT OPTION CLASSIFICATION (post-processing)


def run_roc_postproc(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name):
    base = LogisticRegression(max_iter=300, random_state=SEED)
    base.fit(X_tr, y_tr)
    y_score = base.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    # Build AIF360 datasets
    bld_te_true = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_scores = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)
    # Attach the continuous scores for ROC to use
    bld_te_scores.scores = y_score.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{prot_name: 1}],
        privileged_groups=[{prot_name: 0}],
        metric_name="Statistical parity difference",
        metric_lb=-0.02,
        metric_ub=0.02,
        num_class_thresh=100,
        num_ROC_margin=50
    )
    roc = roc.fit(bld_te_true, bld_te_scores)
    bld_post = roc.predict(bld_te_scores)

    # Extract for performance
    y_pred_post = bld_post.labels.ravel().astype(int)
    y_score_post = getattr(bld_post, 'scores', None)
    if y_score_post is None:
        y_score_post = y_score

    perf = compute_performance(y_te, y_pred_post, y_score_post)
    fair = fairness_report(bld_te_true, bld_post, prot_name)
    return perf, fair

# 4.5 Run all methods on BOTH datasets and BOTH protected attributes

results = []

def run_all_for_dataset(name, X_tr, y_tr, X_te, y_te, p_tr_age, p_te_age, p_tr_edu, p_te_edu):
    # Age
    perf_rw, fair_rw = run_reweighing(X_tr, y_tr, X_te, y_te, p_tr_age, p_te_age, 'age_bin')
    results.append((name, 'age_bin', 'Reweighing', perf_rw, fair_rw))

    perf_adv, fair_adv = run_adversarial(X_tr, y_tr, X_te, y_te, p_tr_age, p_te_age, 'age_bin', scope=f'adv_{name}_age')
    results.append((name, 'age_bin', 'Adversarial', perf_adv, fair_adv))

    perf_roc, fair_roc = run_roc_postproc(X_tr, y_tr, X_te, y_te, p_tr_age, p_te_age, 'age_bin')
    results.append((name, 'age_bin', 'ROC_post', perf_roc, fair_roc))

    # Education
    perf_rw, fair_rw = run_reweighing(X_tr, y_tr, X_te, y_te, p_tr_edu, p_te_edu, 'edu_bin')
    results.append((name, 'edu_bin', 'Reweighing', perf_rw, fair_rw))

    perf_adv, fair_adv = run_adversarial(X_tr, y_tr, X_te, y_te, p_tr_edu, p_te_edu, 'edu_bin', scope=f'adv_{name}_edu')
    results.append((name, 'edu_bin', 'Adversarial', perf_adv, fair_adv))

    perf_roc, fair_roc = run_roc_postproc(X_tr, y_tr, X_te, y_te, p_tr_edu, p_te_edu, 'edu_bin')
    results.append((name, 'edu_bin', 'ROC_post', perf_roc, fair_roc))


# Adult dataset runs
run_all_for_dataset(
    'Adult',
    X_tr_ad_t, y_tr_ad, X_te_ad_t, y_te_ad,
    p_tr_ad_age, p_te_ad_age,
    p_tr_ad_edu, p_te_ad_edu
)

# Kaggle dataset runs
run_all_for_dataset(
    'Kaggle',
    X_tr_kg_t, y_tr_kg, X_te_kg_t, y_te_kg,
    p_tr_kg_age, p_te_kg_age,
    p_tr_kg_edu, p_te_kg_edu
)

# 4.6 Pretty print results

rows = []
for ds, attr, method, perf, fair in results:
    rows.append({
        'dataset': ds,
        'protected_attr': attr,
        'method': method,
        'accuracy': perf['accuracy'],
        'f1': perf['f1'],
        'auc': perf['auc'],
        'SPD': fair['SPD'], 'EOD': fair['EOD'], 'DI': fair['DI']
    })

res_df = pd.DataFrame(rows)
print_block('DEBIASING COMPARISON RESULTS', res_df)

# Also show grouped summary by dataset/attr/method
def fmt(x):
    return f"{x['mean']:.3f}"
summary = res_df.groupby(['dataset','protected_attr','method']).agg({
    'accuracy':'mean','f1':'mean','auc':'mean','SPD':'mean','EOD':'mean','DI':'mean'
}).reset_index()

print_block('SUMMARY (mean values)', summary)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.742295; batch adversarial loss: 0.741474
epoch 1; iter: 0; batch classifier loss: 0.347234; batch adversarial loss: 0.714793
epoch 2; iter: 0; batch classifier loss: 0.305107; batch adversarial loss: 0.727259
epoch 3; iter: 0; batch classifier loss: 0.288207; batch adversarial loss: 0.700075
epoch 4; iter: 0; batch classifier loss: 0.325124; batch adversarial loss: 0.680498
epoch 5; iter: 0; batch classifier loss: 0.320235; batch adversarial loss: 0.690856
epoch 6; iter: 0; batch classifier loss: 0.356991; batch adversarial loss: 0.674536
epoch 7; iter: 0; batch classifier loss: 0.336697; batch adversarial loss: 0.667821
epoch 8; iter: 0; batch classifier loss: 0.318562; batch adversarial loss: 0.669220
epoch 9; iter: 0; batch classifier loss: 0.380762; batch adversarial loss: 0.689869
epoch 10; iter: 0; batch classifier loss: 0.405593; batch adversarial loss: 0.678475
epoch 11; iter: 0; batch classifier loss: 0.331877; batch adversarial loss:

In [ ]:
# Step 5A (Adult only): Repeated runs + significance tests vs Baseline


import gc


def paired_tests(scores_a, scores_b):
    a = np.array(scores_a, dtype=float)
    b = np.array(scores_b, dtype=float)
    t_p = stats.ttest_rel(a, b, alternative='two-sided').pvalue
    try:
        w_p = stats.wilcoxon(a, b, alternative='two-sided', zero_method='wilcox').pvalue
    except Exception:
        w_p = np.nan
    return dict(ttest_p=t_p, wilcoxon_p=w_p)

def summarize_mean_std(df, group_cols=('dataset','protected_attr','method')):
    agg = df.groupby(list(group_cols)).agg(
        accuracy_mean=('accuracy','mean'), accuracy_std=('accuracy','std'),
        f1_mean=('f1','mean'), f1_std=('f1','std'),
        auc_mean=('auc','mean'), auc_std=('auc','std'),
        SPD_mean=('SPD','mean'), SPD_std=('SPD','std'),
        EOD_mean=('EOD','mean'), EOD_std=('EOD','std'),
        DI_mean=('DI','mean'), DI_std=('DI','std'),
        runs=('accuracy','count')
    ).reset_index()
    return agg

def print_block(title, obj):
    print('\n' + '='*100)
    print(title)
    print('='*100)
    print(obj)

# Local runners

def s5_run_baseline(X_tr, y_tr, X_te, y_te, prot_te, prot_name, seed=42):
    clf = LogisticRegression(max_iter=300, random_state=seed)
    clf.fit(X_tr, y_tr)
    y_score = clf.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    bld_te_true = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_pred = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te_true, bld_te_pred, prot_name)
    return perf, fair

def s5_run_reweighing(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=42):
    bld_tr = to_bld_from_arrays(X_tr, y_tr, prot_tr, prot_name=prot_name)
    rw = Reweighing(
        unprivileged_groups=[{prot_name: 1}],
        privileged_groups=[{prot_name: 0}]
    )
    bld_tr_rw = rw.fit_transform(bld_tr)
    sample_w = bld_tr_rw.instance_weights

    clf = LogisticRegression(max_iter=300, random_state=seed)
    clf.fit(X_tr, y_tr, sample_weight=sample_w)
    y_score = clf.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    bld_te_true = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_pred = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te_true, bld_te_pred, prot_name)
    return perf, fair

def s5_run_adversarial(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, scope, seed=42, epochs=30):
    bld_tr = to_bld_from_arrays(X_tr, y_tr, prot_tr, prot_name=prot_name)
    bld_te = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)

    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

    adv = AdversarialDebiasing(
        privileged_groups=[{prot_name: 0}],
        unprivileged_groups=[{prot_name: 1}],
        scope_name=scope,
        debias=True,
        sess=sess,
        num_epochs=30,
        batch_size=256,
        classifier_num_hidden_units=64
    )
    adv.fit(bld_tr)
    bld_pred = adv.predict(bld_te)

    y_pred = bld_pred.labels.ravel().astype(int)
    y_score = bld_pred.scores.ravel()

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te, bld_pred, prot_name)

    sess.close()
    tf.compat.v1.reset_default_graph()
    gc.collect()
    return perf, fair

def s5_run_roc_postproc(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=42):
    base = LogisticRegression(max_iter=300, random_state=seed)
    base.fit(X_tr, y_tr)
    y_score = base.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    bld_te_true   = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_scores = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)
    bld_te_scores.scores = y_score.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{prot_name: 1}],
        privileged_groups=[{prot_name: 0}],
        metric_name="Statistical parity difference",
        metric_lb=-0.02, metric_ub=0.02,
        num_class_thresh=100, num_ROC_margin=50
    )
    roc = roc.fit(bld_te_true, bld_te_scores)
    bld_post = roc.predict(bld_te_scores)

    y_pred_post = bld_post.labels.ravel().astype(int)
    y_score_post = getattr(bld_post, 'scores', None)
    if y_score_post is None:
        y_score_post = y_score

    perf = compute_performance(y_te, y_pred_post, y_score_post)
    fair = fairness_report(bld_te_true, bld_post, prot_name)
    return perf, fair

def run_once_for_attr(dataset_name, X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=42):
    out = []

    perf, fair = s5_run_baseline(X_tr, y_tr, X_te, y_te, prot_te, prot_name, seed=seed)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='Baseline', **perf, **fair))

    perf, fair = s5_run_reweighing(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=seed)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='Reweighing', **perf, **fair))

    scope = f"adv_{dataset_name}_{prot_name}_seed{seed}"
    perf, fair = s5_run_adversarial(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, scope=scope, seed=seed, epochs=30)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='Adversarial', **perf, **fair))

    perf, fair = s5_run_roc_postproc(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=seed)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='ROC_post', **perf, **fair))

    return out

# Adult only repeated runs

n_runs = 5  # reduce to 3 if RAM is tight
all_rows = []

for r in range(n_runs):
    seed = 42 + r

    # Adult - age_bin
    all_rows += run_once_for_attr(
        'Adult',
        X_tr_ad_t, y_tr_ad, X_te_ad_t, y_te_ad,
        p_tr_ad_age, p_te_ad_age,
        'age_bin',
        seed=seed
    )

    # Adult - edu_bin
    all_rows += run_once_for_attr(
        'Adult',
        X_tr_ad_t, y_tr_ad, X_te_ad_t, y_te_ad,
        p_tr_ad_edu, p_te_ad_edu,
        'edu_bin',
        seed=seed
    )

res5_adult = pd.DataFrame(all_rows)
print_block('STEP 5A — ADULT RAW (head)', res5_adult.head())
print_block('ADULT Results shape', res5_adult.shape)

summary5_adult = summarize_mean_std(res5_adult, group_cols=('dataset','protected_attr','method'))
print_block('STEP 5A — ADULT MEAN ± STD', summary5_adult)

metrics = ['accuracy', 'f1', 'auc', 'SPD', 'EOD', 'DI']
methods_to_compare = ['Reweighing', 'Adversarial', 'ROC_post']

pv_rows = []
for (ds, attr), grp in res5_adult.groupby(['dataset','protected_attr']):
    base = grp[grp['method'] == 'Baseline']
    for m in methods_to_compare:
        comp = grp[grp['method'] == m]
        for metric in metrics:
            base_vals = base[metric].values
            comp_vals = comp[metric].values
            if len(base_vals) == len(comp_vals) and len(base_vals) > 1:
                pvals = paired_tests(base_vals, comp_vals)
                pv_rows.append({
                    'dataset': ds,
                    'protected_attr': attr,
                    'method_vs_baseline': m,
                    'metric': metric,
                    'ttest_p': pvals['ttest_p'],
                    'wilcoxon_p': pvals['wilcoxon_p']
                })

pvals5_adult = pd.DataFrame(pv_rows)
if not pvals5_adult.empty:
    pvals5_adult['sig_ttest@0.05'] = pvals5_adult['ttest_p'] < 0.05
    pvals5_adult['sig_wilcoxon@0.05'] = pvals5_adult['wilcoxon_p'] < 0.05

print_block('STEP 5A — ADULT SIGNIFICANCE', pvals5_adult)

# Save artifacts to disk
res5_adult.to_csv('step5_adult_raw.csv', index=False)
summary5_adult.to_csv('step5_adult_summary.csv', index=False)
pvals5_adult.to_csv('step5_adult_pvals.csv', index=False)

print('\nSaved: step5_adult_raw.csv, step5_adult_summary.csv, step5_adult_pvals.csv')


epoch 0; iter: 0; batch classifier loss: 0.742295; batch adversarial loss: 0.741474
epoch 1; iter: 0; batch classifier loss: 0.347234; batch adversarial loss: 0.714793
epoch 2; iter: 0; batch classifier loss: 0.305107; batch adversarial loss: 0.727259
epoch 3; iter: 0; batch classifier loss: 0.288207; batch adversarial loss: 0.700075
epoch 4; iter: 0; batch classifier loss: 0.325124; batch adversarial loss: 0.680498
epoch 5; iter: 0; batch classifier loss: 0.320235; batch adversarial loss: 0.690856
epoch 6; iter: 0; batch classifier loss: 0.356991; batch adversarial loss: 0.674536
epoch 7; iter: 0; batch classifier loss: 0.336697; batch adversarial loss: 0.667821
epoch 8; iter: 0; batch classifier loss: 0.318562; batch adversarial loss: 0.669220
epoch 9; iter: 0; batch classifier loss: 0.380762; batch adversarial loss: 0.689869
epoch 10; iter: 0; batch classifier loss: 0.405593; batch adversarial loss: 0.678475
epoch 11; iter: 0; batch classifier loss: 0.331877; batch adversarial loss:

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [ ]:

!ls -lh *.csv

from google.colab import files
files.download("step5_adult_raw.csv")
files.download("step5_adult_summary.csv")
files.download("step5_adult_pvals.csv")



-rw-r--r-- 1 root root 1.7M Aug 10 17:15 adult_test.csv
-rw-r--r-- 1 root root 3.4M Aug 10 17:15 adult_train.csv
-rw-r--r-- 1 root root  63K Aug 10 17:15 recruitment_data.csv
-rw-r--r-- 1 root root 2.1K Aug 10 18:08 step5_adult_pvals.csv
-rw-r--r-- 1 root root 5.6K Aug 10 18:08 step5_adult_raw.csv
-rw-r--r-- 1 root root 1.7K Aug 10 18:08 step5_adult_summary.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# Step 5B (Kaggle): Repeated runs + significance tests vs Baseline



import gc



def paired_tests(scores_a, scores_b):
    a = np.array(scores_a, dtype=float)
    b = np.array(scores_b, dtype=float)
    t_p = stats.ttest_rel(a, b, alternative='two-sided').pvalue
    try:
        w_p = stats.wilcoxon(a, b, alternative='two-sided', zero_method='wilcox').pvalue
    except Exception:
        w_p = np.nan
    return dict(ttest_p=t_p, wilcoxon_p=w_p)

def summarize_mean_std(df, group_cols=('dataset','protected_attr','method')):
    agg = df.groupby(list(group_cols)).agg(
        accuracy_mean=('accuracy','mean'), accuracy_std=('accuracy','std'),
        f1_mean=('f1','mean'), f1_std=('f1','std'),
        auc_mean=('auc','mean'), auc_std=('auc','std'),
        SPD_mean=('SPD','mean'), SPD_std=('SPD','std'),
        EOD_mean=('EOD','mean'), EOD_std=('EOD','std'),
        DI_mean=('DI','mean'), DI_std=('DI','std'),
        runs=('accuracy','count')
    ).reset_index()
    return agg

def print_block(title, obj):
    print('\n' + '='*100)
    print(title)
    print('='*100)
    print(obj)

# Local runners

def s5_run_baseline(X_tr, y_tr, X_te, y_te, prot_te, prot_name, seed=42):
    clf = LogisticRegression(max_iter=300, random_state=seed)
    clf.fit(X_tr, y_tr)
    y_score = clf.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    bld_te_true = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_pred = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te_true, bld_te_pred, prot_name)
    return perf, fair

def s5_run_reweighing(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=42):
    bld_tr = to_bld_from_arrays(X_tr, y_tr, prot_tr, prot_name=prot_name)
    rw = Reweighing(
        unprivileged_groups=[{prot_name: 1}],
        privileged_groups=[{prot_name: 0}]
    )
    bld_tr_rw = rw.fit_transform(bld_tr)
    sample_w = bld_tr_rw.instance_weights

    clf = LogisticRegression(max_iter=300, random_state=seed)
    clf.fit(X_tr, y_tr, sample_weight=sample_w)
    y_score = clf.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    bld_te_true = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_pred = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te_true, bld_te_pred, prot_name)
    return perf, fair

def s5_run_adversarial(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, scope, seed=42, epochs=30):
    bld_tr = to_bld_from_arrays(X_tr, y_tr, prot_tr, prot_name=prot_name)
    bld_te = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)

    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

    adv = AdversarialDebiasing(
        privileged_groups=[{prot_name: 0}],
        unprivileged_groups=[{prot_name: 1}],
        scope_name=scope,
        debias=True,
        sess=sess,
        num_epochs=30,            # keep lighter than 50
        batch_size=256,
        classifier_num_hidden_units=64
    )
    adv.fit(bld_tr)
    bld_pred = adv.predict(bld_te)

    y_pred = bld_pred.labels.ravel().astype(int)
    y_score = bld_pred.scores.ravel()

    perf = compute_performance(y_te, y_pred, y_score)
    fair = fairness_report(bld_te, bld_pred, prot_name)

    sess.close()
    tf.compat.v1.reset_default_graph()
    gc.collect()
    return perf, fair

def s5_run_roc_postproc(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=42):
    base = LogisticRegression(max_iter=300, random_state=seed)
    base.fit(X_tr, y_tr)
    y_score = base.predict_proba(X_te)[:, 1]
    y_pred  = (y_score >= 0.5).astype(int)

    bld_te_true   = to_bld_from_arrays(X_te, y_te, prot_te, prot_name=prot_name)
    bld_te_scores = to_bld_from_arrays(X_te, y_pred, prot_te, prot_name=prot_name)
    bld_te_scores.scores = y_score.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{prot_name: 1}],
        privileged_groups=[{prot_name: 0}],
        metric_name="Statistical parity difference",
        metric_lb=-0.02, metric_ub=0.02,
        num_class_thresh=100, num_ROC_margin=50
    )
    roc = roc.fit(bld_te_true, bld_te_scores)
    bld_post = roc.predict(bld_te_scores)

    y_pred_post = bld_post.labels.ravel().astype(int)
    y_score_post = getattr(bld_post, 'scores', None)
    if y_score_post is None:
        y_score_post = y_score

    perf = compute_performance(y_te, y_pred_post, y_score_post)
    fair = fairness_report(bld_te_true, bld_post, prot_name)
    return perf, fair

def run_once_for_attr(dataset_name, X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=42):
    out = []
    perf, fair = s5_run_baseline(X_tr, y_tr, X_te, y_te, prot_te, prot_name, seed=seed)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='Baseline', **perf, **fair))

    perf, fair = s5_run_reweighing(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=seed)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='Reweighing', **perf, **fair))

    scope = f"adv_{dataset_name}_{prot_name}_seed{seed}"
    perf, fair = s5_run_adversarial(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, scope=scope, seed=seed, epochs=30)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='Adversarial', **perf, **fair))

    perf, fair = s5_run_roc_postproc(X_tr, y_tr, X_te, y_te, prot_tr, prot_te, prot_name, seed=seed)
    out.append(dict(dataset=dataset_name, protected_attr=prot_name, method='ROC_post', **perf, **fair))
    return out

# Kaggle only repeated runs

n_runs = 5
all_rows = []

for r in range(n_runs):
    seed = 42 + r

    # Kaggle - age_bin
    all_rows += run_once_for_attr(
        'Kaggle',
        X_tr_kg_t, y_tr_kg, X_te_kg_t, y_te_kg,
        p_tr_kg_age, p_te_kg_age,
        'age_bin',
        seed=seed
    )

    # Kaggle - edu_bin
    all_rows += run_once_for_attr(
        'Kaggle',
        X_tr_kg_t, y_tr_kg, X_te_kg_t, y_te_kg,
        p_tr_kg_edu, p_te_kg_edu,
        'edu_bin',
        seed=seed
    )

res5_kaggle = pd.DataFrame(all_rows)
print_block('STEP 5B — KAGGLE RAW (head)', res5_kaggle.head())
print_block('KAGGLE Results shape', res5_kaggle.shape)

summary5_kaggle = summarize_mean_std(res5_kaggle, group_cols=('dataset','protected_attr','method'))
print_block('STEP 5B — KAGGLE MEAN ± STD', summary5_kaggle)

metrics = ['accuracy', 'f1', 'auc', 'SPD', 'EOD', 'DI']
methods_to_compare = ['Reweighing', 'Adversarial', 'ROC_post']

pv_rows = []
for (ds, attr), grp in res5_kaggle.groupby(['dataset','protected_attr']):
    base = grp[grp['method'] == 'Baseline']
    for m in methods_to_compare:
        comp = grp[grp['method'] == m]
        for metric in metrics:
            base_vals = base[metric].values
            comp_vals = comp[metric].values
            if len(base_vals) == len(comp_vals) and len(base_vals) > 1:
                pvals = paired_tests(base_vals, comp_vals)
                pv_rows.append({
                    'dataset': ds,
                    'protected_attr': attr,
                    'method_vs_baseline': m,
                    'metric': metric,
                    'ttest_p': pvals['ttest_p'],
                    'wilcoxon_p': pvals['wilcoxon_p']
                })

pvals5_kaggle = pd.DataFrame(pv_rows)
if not pvals5_kaggle.empty:
    pvals5_kaggle['sig_ttest@0.05'] = pvals5_kaggle['ttest_p'] < 0.05
    pvals5_kaggle['sig_wilcoxon@0.05'] = pvals5_kaggle['wilcoxon_p'] < 0.05

print_block('STEP 5B — KAGGLE SIGNIFICANCE', pvals5_kaggle)

# Save artifacts to disk
res5_kaggle.to_csv('step5_kaggle_raw.csv', index=False)
summary5_kaggle.to_csv('step5_kaggle_summary.csv', index=False)
pvals5_kaggle.to_csv('step5_kaggle_pvals.csv', index=False)

print('\nSaved: step5_kaggle_raw.csv, step5_kaggle_summary.csv, step5_kaggle_pvals.csv')


epoch 0; iter: 0; batch classifier loss: 0.896412; batch adversarial loss: 0.696458
epoch 1; iter: 0; batch classifier loss: 0.843614; batch adversarial loss: 0.670863
epoch 2; iter: 0; batch classifier loss: 0.864863; batch adversarial loss: 0.660115
epoch 3; iter: 0; batch classifier loss: 0.810308; batch adversarial loss: 0.660333
epoch 4; iter: 0; batch classifier loss: 0.758222; batch adversarial loss: 0.663698
epoch 5; iter: 0; batch classifier loss: 0.724666; batch adversarial loss: 0.679846
epoch 6; iter: 0; batch classifier loss: 0.697580; batch adversarial loss: 0.682563
epoch 7; iter: 0; batch classifier loss: 0.710323; batch adversarial loss: 0.649437
epoch 8; iter: 0; batch classifier loss: 0.662141; batch adversarial loss: 0.698299
epoch 9; iter: 0; batch classifier loss: 0.646967; batch adversarial loss: 0.676430
epoch 10; iter: 0; batch classifier loss: 0.629738; batch adversarial loss: 0.679103
epoch 11; iter: 0; batch classifier loss: 0.611379; batch adversarial loss:

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [ ]:
!ls -lh *.csv

from google.colab import files
files.download("step5_kaggle_raw.csv")
files.download("step5_kaggle_summary.csv")
files.download("step5_kaggle_pvals.csv")

-rw-r--r-- 1 root root 1.7M Aug 10 18:14 'adult_test (1).csv'
-rw-r--r-- 1 root root 1.7M Aug 10 17:15  adult_test.csv
-rw-r--r-- 1 root root 3.4M Aug 10 18:14 'adult_train (1).csv'
-rw-r--r-- 1 root root 3.4M Aug 10 17:15  adult_train.csv
-rw-r--r-- 1 root root  63K Aug 10 18:14 'recruitment_data (1).csv'
-rw-r--r-- 1 root root  63K Aug 10 17:15  recruitment_data.csv
-rw-r--r-- 1 root root 2.1K Aug 10 18:08  step5_adult_pvals.csv
-rw-r--r-- 1 root root 5.6K Aug 10 18:08  step5_adult_raw.csv
-rw-r--r-- 1 root root 1.7K Aug 10 18:08  step5_adult_summary.csv
-rw-r--r-- 1 root root 2.2K Aug 10 18:18  step5_kaggle_pvals.csv
-rw-r--r-- 1 root root 5.6K Aug 10 18:18  step5_kaggle_raw.csv
-rw-r--r-- 1 root root 1.7K Aug 10 18:18  step5_kaggle_summary.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Results Visualization

Grouped Histograms (Performance & Fairness)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


adult_perf = {"Accuracy": 0.853, "F1": 0.658, "AUC": 0.905}
kaggle_perf = {"Accuracy": 0.857, "F1": 0.751, "AUC": 0.891}


adult_age = {"SPD": -0.186, "EOD": -0.129, "DI": 0.376}
kaggle_age = {"SPD":  0.006, "EOD": -0.051, "DI": 1.023}


adult_edu = {"SPD": -0.434, "EOD": -0.479, "DI": 0.165}
kaggle_edu = {"SPD": -0.264, "EOD": -0.162, "DI": 0.401}


plt.rcParams.update({"figure.dpi": 160})
fig = plt.figure(figsize=(12, 5))


ax1 = fig.add_subplot(1, 2, 1)
metrics_perf = list(adult_perf.keys())
x = np.arange(len(metrics_perf))
w = 0.35

adult_vals = [adult_perf[m] for m in metrics_perf]
kaggle_vals = [kaggle_perf[m] for m in metrics_perf]

b1 = ax1.bar(x - w/2, adult_vals, width=w, label="Adult")
b2 = ax1.bar(x + w/2, kaggle_vals, width=w, label="Kaggle")

ax1.set_xticks(x)
ax1.set_xticklabels(metrics_perf)
ax1.set_ylim(0, 1.05)
ax1.set_ylabel("Score")
ax1.set_title("Baseline Performance (no debiasing)")
ax1.legend()


def annotate_bars(ax, bars, fmt="{:.3f}"):
    for bar in bars:
        h = bar.get_height()
        ax.annotate(fmt.format(h),
                    xy=(bar.get_x() + bar.get_width()/2, h),
                    xytext=(0, 3), textcoords="offset points",
                    ha="center", va="bottom", fontsize=8)

annotate_bars(ax1, b1)
annotate_bars(ax1, b2)


ax2 = fig.add_subplot(1, 2, 2)
fair_metrics = ["SPD", "EOD", "DI"]


cats = [f"{m}(Age)" for m in fair_metrics] + [f"{m}(Edu)" for m in fair_metrics]
x2 = np.arange(len(cats))
w2 = 0.35

adult_vals2 = [adult_age[m] for m in fair_metrics] + [adult_edu[m] for m in fair_metrics]
kaggle_vals2 = [kaggle_age[m] for m in fair_metrics] + [kaggle_edu[m] for m in fair_metrics]

b3 = ax2.bar(x2 - w2/2, adult_vals2, width=w2, label="Adult")
b4 = ax2.bar(x2 + w2/2, kaggle_vals2, width=w2, label="Kaggle")

ax2.set_xticks(x2)
ax2.set_xticklabels(cats, rotation=20)
ax2.axhline(0, linestyle="--", linewidth=1)
ax2.set_ylabel("Value")
ax2.set_title("Baseline Fairness Metrics by Attribute")
ax2.legend()

annotate_bars(ax2, b3)
annotate_bars(ax2, b4)

plt.tight_layout()
plt.savefig("baseline_bars.png", bbox_inches="tight")
plt.close()

print("Saved: baseline_bars.png")


Saved: baseline_bars.png


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({
    "figure.dpi": 160,
    "axes.unicode_minus": False
})

baseline_perf = {
    "Adult":   {"Accuracy": 0.853, "F1": 0.658, "AUC": 0.905},
    "Kaggle":  {"Accuracy": 0.857, "F1": 0.751, "AUC": 0.891},
}

baseline_fair = {
    ("Adult",  "age"): {"SPD": -0.186, "EOD": -0.129, "DI": 0.376},
    ("Adult",  "edu"): {"SPD": -0.434, "EOD": -0.479, "DI": 0.165},
    ("Kaggle", "age"): {"SPD":  0.006, "EOD": -0.051, "DI": 1.023},
    ("Kaggle", "edu"): {"SPD": -0.264, "EOD": -0.162, "DI": 0.401},
}

methods = ["Baseline", "Reweighing", "Adversarial", "ROC_post"]

perf = {
    ("Adult",  "age"): {
        "Baseline":   baseline_perf["Adult"],
        "Reweighing": {"Accuracy": 0.850772, "F1": 0.646459, "AUC": 0.899566},
        "Adversarial":{"Accuracy": 0.848988, "F1": 0.641134, "AUC": 0.903116},
        "ROC_post":   {"Accuracy": 0.777573, "F1": 0.635851, "AUC": 0.905339},
    },
    ("Adult",  "edu"): {
        "Baseline":   baseline_perf["Adult"],
        "Reweighing": {"Accuracy": 0.831088, "F1": 0.596651, "AUC": 0.884159},
        "Adversarial":{"Accuracy": 0.841238, "F1": 0.626537, "AUC": 0.900191},
        "ROC_post":   {"Accuracy": 0.764286, "F1": 0.611043, "AUC": 0.905339},
    },
    ("Kaggle", "age"): {
        "Baseline":   baseline_perf["Kaggle"],
        "Reweighing": {"Accuracy": 0.853333, "F1": 0.747126, "AUC": 0.891642},
        "Adversarial":{"Accuracy": 0.846667, "F1": 0.722892, "AUC": 0.884525},
        "ROC_post":   {"Accuracy": 0.846667, "F1": 0.757895, "AUC": 0.891123},
    },
    ("Kaggle", "edu"): {
        "Baseline":   baseline_perf["Kaggle"],
        "Reweighing": {"Accuracy": 0.866667, "F1": 0.761905, "AUC": 0.887486},
        "Adversarial":{"Accuracy": 0.860000, "F1": 0.740741, "AUC": 0.889201},
        "ROC_post":   {"Accuracy": 0.820000, "F1": 0.727273, "AUC": 0.891123},
    },
}

fair = {
    ("Adult",  "age"): {
        "Baseline":   baseline_fair[("Adult","age")],
        "Reweighing": {"SPD": -0.114940, "EOD": -0.000812, "DI": 0.540433},
        "Adversarial":{"SPD": -0.066077, "EOD":  0.110919, "DI": 0.701633},
        "ROC_post":   {"SPD": -0.018674, "EOD":  0.176346, "DI": 0.951488},
    },
    ("Adult",  "edu"): {
        "Baseline":   baseline_fair[("Adult","edu")],
        "Reweighing": {"SPD": -0.125792, "EOD":  0.025151, "DI": 0.545889},
        "Adversarial":{"SPD": -0.186249, "EOD": -0.057537, "DI": 0.433532},
        "ROC_post":   {"SPD": -0.019757, "EOD":  0.227597, "DI": 0.948621},
    },
    ("Kaggle", "age"): {
        "Baseline":   baseline_fair[("Kaggle","age")],
        "Reweighing": {"SPD":  0.011006, "EOD": -0.050794, "DI": 1.041906},
        "Adversarial":{"SPD":  0.061661, "EOD":  0.017460, "DI": 1.305224},
        "ROC_post":   {"SPD":  0.015227, "EOD":  0.011111, "DI": 1.048628},
    },
    ("Kaggle", "edu"): {
        "Baseline":   baseline_fair[("Kaggle","edu")],
        "Reweighing": {"SPD": -0.141117, "EOD": -0.012059, "DI": 0.588745},
        "Adversarial":{"SPD": -0.126857, "EOD":  0.026438, "DI": 0.595644},
        "ROC_post":   {"SPD": -0.019311, "EOD":  0.083488, "DI": 0.946765},
    },
}

def ensure_dir(path="figs"):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def annotate_bars(ax, bars, fmt="{:.3f}"):
    for bar in bars:
        h = bar.get_height()
        ax.annotate(fmt.format(h),
                    xy=(bar.get_x() + bar.get_width()/2, h),
                    xytext=(0, 3), textcoords="offset points",
                    ha="center", va="bottom", fontsize=8)

def plot_grouped_bars(metric_names, data_by_method, title, ylabel, save_path, ylim=None, rotate=0):
    x = np.arange(len(metric_names))
    group_w = 0.85
    n = len(methods)
    bar_w = group_w / n
    offset = -(group_w - bar_w)/2
    fig = plt.figure(figsize=(6.8, 4.5))
    ax = fig.add_subplot(1,1,1)
    for i, m in enumerate(methods):
        vals = [data_by_method[m][mn] for mn in metric_names]
        b = ax.bar(x + offset + i*bar_w, vals, width=bar_w, label=m)
        annotate_bars(ax, b)
    ax.set_xticks(x)
    ax.set_xticklabels(metric_names, rotation=rotate)
    ax.set_ylabel(ylabel)
    ax.set_title(title, pad=10)
    if ylim is not None:
        ax.set_ylim(*ylim)
    ax.axhline(0, linestyle="--", linewidth=1)
    ax.legend(ncol=2, fontsize=9, frameon=True)
    fig.tight_layout()
    fig.savefig(save_path, bbox_inches="tight")
    plt.close(fig)

outdir = ensure_dir("figs")
perf_metrics = ["Accuracy", "F1", "AUC"]
fair_metrics = ["SPD", "EOD", "DI"]
perf_ylim = (0.0, 1.05)
fair_ylim = (-0.6, 1.4)
labels_map = {"age": "Binned Age", "edu": "Binned Education"}

for ds in ["Adult", "Kaggle"]:
    for attr in ["age", "edu"]:
        title_p = f"{ds} – {labels_map[attr]}: Performance comparison (Accuracy / F1 / AUC)"
        file_p  = os.path.join(outdir, f"fig4_3_performance_{ds.lower()}_{attr}.png")
        plot_grouped_bars(perf_metrics, perf[(ds, attr)], title_p, ylabel="Score", save_path=file_p, ylim=perf_ylim, rotate=0)
        title_f = f"{ds} – {labels_map[attr]}: Fairness comparison (SPD / EOD / DI)"
        file_f  = os.path.join(outdir, f"fig4_3_fairness_{ds.lower()}_{attr}.png")
        plot_grouped_bars(fair_metrics, fair[(ds, attr)], title_f, ylabel="Value", save_path=file_f, ylim=fair_ylim, rotate=0)

print("All figures saved to:", outdir)



All figures saved to: figs
